# Venting

## Overview

Copyright 2024 Moran Innovation LLC

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.

CryoFM™ is a trademark of Moran Innovation LLC. All rights reserved.

### Usage Notes

**HOW TO USE THIS INTERACTIVE NOTEBOOK:**
* Use the table of contents icon and sidebar at the left to navigate to the topics of interest.
* Change any of the input values and run the associated cell code.
* If an interactive Binder web browser session is being used, the original file will not be altered. Therefore, if you wish to retain any modifications you make, save your file to local memory on your device.

**CAUTION:** None of the code cells run automatically. The user must manually run selected code cells in sequence to achieve the desired results. Default units are SI.

**REFERENCE REPORT AND TRAINING:** The reference report for this interactive notebook is "Cryogenic fluid management of liquid hydrogen, oxygen, and methane: Part 1 - passive technologies, systems, and operations",  Moran Innovation LLC, 2023. An online version of the report and training videos based on the content can be found here: https://sites.google.com/view/matthewemoran/training#h.ir8xm9d8wn6h.

## Vent Losses

### Subcooled Venting

Assumes vent time is short and environmental heating is low enough to ignore.
Otherwise, liquid sensible heating from environment heat leak must be accounted for.
Other assumptions: bulk liquid temperature is constant; interfacial mass transfer
can be neglected; liquid phase can be treated as incompressible.

In [10]:
import sys
import CoolProp.CoolProp as cp

### INPUTS ###

# Initial and final conditions
fluid = 'Oxygen'
mass_liq_init = 8.5606e4  # kg, initial liquid mass
volume_tank = 1.505e2  # m^3, tank volume
press_init = 3.0e5  # Pa, initial tank pressure
temp_liq_init = 91.0  # K, initial bulk liquid temperature
temp_vap_init = 250.0  # K, initial bulk ullage vapor temperature
press_final = 1.5e5  # Pa, final tank pressure after venting

# Fluid properties
dens_liq_init = cp.PropsSI('D', 'P', press_init, 'T', temp_liq_init, fluid)
dens_vap_init = cp.PropsSI('D', 'P', press_init, 'T', temp_vap_init, fluid)
temp_sat_final = cp.PropsSI('T', 'P', press_final, 'Q', 0, fluid)
press_sat_liq = cp.PropsSI('P', 'T', temp_liq_init, 'Q', 0, fluid)
if fluid == 'Parahydrogen' or fluid == 'Hydrogen':
    spheat_ratio = 1.41
elif fluid == 'Oxygen':
    spheat_ratio = 1.40
elif fluid == 'Methane':
    spheat_ratio = 1.32
else:
    print("Supported fluids are parahydrogen, hydrogen, oxygen, or methane")

### OUTPUTS ###

# Check for subcooled vs saturated venting
if temp_sat_final < temp_liq_init:
    print("Venting is subcooled only to", "{:.3e}".format(press_sat_liq), "Pa")
    print("Adjust the inputs for the subcooled venting pressure range")
    print("and use saturated venting algorithm for remaining pressure range")
    sys.exit("The liquid is not subcooled through the entire venting process")

# Volumetric fill level (initial)
fill_init = mass_liq_init / volume_tank / dens_liq_init

# Ullage vapor mass (initial; single constituent ullage)
mass_vap_init = dens_vap_init * volume_tank * (1. - fill_init)

# Final ullage temperature based on adiabatic (polytropic) expansion
temp_vap_final = temp_vap_init * (press_final / press_init)**((spheat_ratio - 1)
                                                             /spheat_ratio)

# Final ullage mass
dens_vap_final = cp.PropsSI('D', 'P', press_final, 'T', temp_vap_final, fluid)
mass_vap_final = dens_vap_final * volume_tank * (1 - fill_init)

# Mass vented
mass_vented = mass_vap_init - mass_vap_final
print("Vented mass:","{:.3e}".format(mass_vented), "kg")


Vented mass: 1.363e+02 kg
